# The `__set_name__` Method 

- Python 3.6 and above
- Called once when the descriptor is first instantiated 
- Commonly used in error messages

The following example/strategy only works if we're __not__ using `__slots__`

In [1]:
class ValidString:
    def __set_name__(self, owner_class, property_name):
        print(f'__set_name__: owner={owner_class}, property_name={property_name}')

In [2]:
class Person:
    name = ValidString()

__set_name__: owner=<class '__main__.Person'>, property_name=name


Create when the class is *created*, not instanciated (called)

In [3]:
class ValidString:
    def __set_name__(self, owner_class, property_name):
        print(f'__set_name__: owner={owner_class}, property_name={property_name}')
        self.property_name = property_name

    def __get__(self, instance, owner_class):
        if instance is None:
            return self 
        print(f'__get__ was called for property: {self.property_name} of instance {instance}')

In [4]:
class Person:
    first_name = ValidString()
    last_name = ValidString()

__set_name__: owner=<class '__main__.Person'>, property_name=first_name
__set_name__: owner=<class '__main__.Person'>, property_name=last_name


In [7]:
p = Person()
p.first_name

__get__ was called for property: first_name of instance <__main__.Person object at 0x000001DFCC551810>


In [15]:
class ValidString:
    def __init__(self, min_lenght):
        self.min_lenght = min_lenght

    def __set_name__(self, owner_class, property_name):
        print(f'__set_name__: owner={owner_class}, property_name={property_name}')
        self.property_name = property_name

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.property_name} must be a String')
        
        if self.min_lenght is not None and len(value) < self.min_lenght:
            raise ValueError(f'{self.property_name} bust be at least {self.min_lenght} characters.')
        
        key = '_' + self.property_name 
        setattr(instance, key, value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self 
        # print(f'__get__ was called for property: {self.property_name} of instance {instance}')
        key = '_' + self.property_name
        return getattr(instance, key, None)

The classe above still have the issue of constructing the `key` parameter, that can override something sometiem

In [16]:
class Person:
    first_name = ValidString(min_lenght=1)
    last_name = ValidString(min_lenght=2)

__set_name__: owner=<class '__main__.Person'>, property_name=first_name
__set_name__: owner=<class '__main__.Person'>, property_name=last_name


In [18]:
p = Person()

try:
    p.first_name = 'Alex'
    p.last_name = 'M'
except ValueError as ex:
    print(ex)

last_name bust be at least 2 characters.


In [19]:
p.first_name = 'Alezx'
p.last_name = 'Martell'

In [20]:
p.__dict__

{'_first_name': 'Alezx', '_last_name': 'Martell'}

The user still can easily override the stored information

In [27]:
p = Person()
p._first_name = 'Testando'

In [22]:
p.__dict__

{'_first_name': 'Testando'}

In [30]:
p.first_name = 'Gabriel'

In [31]:
p.__dict__

{'_first_name': 'Gabriel'}

The problem is that we're trying to store the attribute value under the same name than the property name that we're storing in the class 

In [32]:
class BankAccount:
    apr = 10

b = BankAccount()

In [33]:
b.apr, b.__dict__

(10, {})

In [34]:
b.apr = 100 
b.__dict__

{'apr': 100}

In [35]:
b.apr

100

This problem depends if we're using data descriptors or non-data descriptors

In [54]:
class ValidString:
    def __init__(self, min_lenght):
        self.min_lenght = min_lenght

    def __set_name__(self, owner_class, property_name):
        print(f'__set_name__: owner={owner_class}, property_name={property_name}')
        self.property_name = property_name

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.property_name} must be a String')
        
        if self.min_lenght is not None and len(value) < self.min_lenght:
            raise ValueError(f'{self.property_name} bust be at least {self.min_lenght} characters.')
       
        # uses the instance dictionary directly 
        instance.__dict__[self.property_name] = value 
        
        ## The approach 
        ### setattr(instance, self.property_name, value)
        ### creates a infinite recursion bc is basically:
        ### instance.property_name = value
        ### That calls the __set__ method 

    def __get__(self, instance, owner_class):
        if instance is None:
            return self 
        print(f'__get__ called')
        return instance.__dict__.get(self.property_name, None)# uses the instance dictionary directly 

In [55]:
class Person:
    first_name = ValidString(2)
    last_name = ValidString(3)

__set_name__: owner=<class '__main__.Person'>, property_name=first_name
__set_name__: owner=<class '__main__.Person'>, property_name=last_name


In [56]:
p1 = Person()
p1.__dict__

{}

In [59]:
p1.first_name = 'Junior'
p1.__dict__

{'first_name': 'Junior'}

In [60]:
p1.first_name

__get__ called


'Junior'